In [1]:
import torch
import math
#torch.set_grad_enabled(False)

In [2]:
#INTERESTING FACT: WHEN COMPUTING THE BATCH GRADIENT PYTORCH DOES NOT TAKE THE SUM OF THE GRADIENTA OF SINGLE DATA POINTS
#BUT THE MEAN. I LEART AT THE EXPENSE OF ABOUT 1H LOL 

In [3]:
class Parameter():
    def __init__(self):
        self.name = ''
        self.data = None
        self.grad = None

In [4]:
class Module(object):
    def forward (self, *input):
        raise NotImplementedError
        
    def backward ( self , * gradwrtoutput ) :
        raise NotImplementedError
    
    def get_parameters( self ) :
        return []   

In [5]:
class Losses(object):        
    def forward():
        return NotImplementedError
    def backward():
        NotImplementedError

In [6]:
class Optimizer(object):
    def zero_grad(self):
        for parameter in self.param : 
            parameter.grad = 0
            
    def step(self):
        raise NotImplementedError

In [7]:
class SGD(Optimizer):
    # this is a SGD optimizer
    def __init__(self,lr,max_iter, parameters) :  
        super().__init__()
        self.eta = lr
        self.maxStep = max_iter 
        self.param = parameters
        self.number_step = 0

    def step(self): 
        if self.number_step <=self.maxStep:
            for parameter in self.param :
                parameter.data = parameter.data - self.eta * parameter.grad
            self.number_step = self.number_step + 1
        return self.param

In [65]:
class Linear(Module):
    
    def __init__(self, input_dim, out_dim, bias = True):
        super().__init__()
        std = 1/math.sqrt(input_dim)
        self.weight = Parameter()
        self.parameters = []
        
        self.weight.data = torch.rand(out_dim, input_dim)
        self.weight.data = 2*std*self.weight.data - std
        self.weight.name = 'weight'
        self.parameters += [self.weight]
        
        self.with_bias = bias
        if bias :
            self.bias = Parameter()
            self.bias.data = torch.rand(out_dim)
            self.bias.data = 2*std*self.bias.data - std
            self.bias.data = self.bias.data.unsqueeze(0)
            self.bias.name = 'bias'
            self.parameters +=[self.bias]
            
        self.x = None
              
    def forward(self, x):
        self.x = x
        self.batch_size = x.shape[0]
        return self.x.mm(self.weight.data.T) + self.bias.data
        
    def backward(self, prev_grad):
        
        prev_grad = prev_grad.view(self.batch_size, -1, 1)
        print(prev_grad.shape)
        if self.x is None:
            raise CallForwardFirst
        
        if self.weight.grad is None:
            self.weight.grad = torch.zeros_like(self.weight.data)
        
        grad_on_batch = prev_grad.view(self.batch_size, -1, 1)*self.x.view(self.batch_size, 1, -1)
        self.weight.grad += grad_on_batch.mean(0)
        
        if self.with_bias:
            if self.bias.grad is None:
                self.bias.grad = torch.zeros_like(self.bias.data)
            grad_on_batch = prev_grad.view(self.batch_size, -1)
            self.bias.grad += grad_on_batch.mean(0)
        
        #if the output has dimension one, squeezing creates problems
        if prev_grad.shape[1]>1:
            prev_grad = prev_grad.squeeze()
        next_grad = prev_grad@self.weight.data
        return next_grad.squeeze()
    
    def get_parameters(self):
        return self.parameters
    

In [66]:
class Tanh(Module):
    def __init__(self):
        self.x = None
    
    def forward (self, x):
        self.x = x
        return torch.tanh(x)
        
    def backward ( self, prev_grad) :
        if self.x is None:
            raise CallForwardFirst
            
        def d(x):
            return 4 * (x.exp() + x.mul(-1).exp()).pow(-2)
        
        return d(self.x)*prev_grad
    

In [67]:
class MSE(Losses):
    # Attention! Works well only when the vectors provided are of the form [batch_size, vector dimension]
    # Otherwise it doesn know what dimesion to pick for the mean computation
    #I'll fix this later
    def __init__(self):
        self.x = None
    def forward(self, x, t):
        self.x = x
        self.t = t
        return (x - t).pow(2).mean()
    
    def backward(self):
        if self.x == None or self.t == None:
            raise CallForwardFirst
        return 2 * (self.x - self.t)/self.x.shape[1]

In [68]:
class Sequential(object):
    def __init__(self, modules):
        super().__init__()
        self.modules=modules
        self.parameters = []
        for m in self.modules:
            param = m.get_parameters()
            if param:
                self.parameters += param
        
    def forward(self,x):
        for m in self.modules:
            x=m.forward(x)
        return x
    
    def backward(self, loss_grad):
        x = loss_grad
        for m in reversed(self.modules):
            x = m.backward(x)
            
    def get_parameters(self):
        return self.parameters

    def set_parameters(self , params):
        self.parameters = params
        

In [69]:
#check that everything is working with loss backward
x = torch.randn(1, 4, requires_grad = False)
y = torch.randn(1, 4,requires_grad = False)

a = x
a.requires_grad_(True)
print(a.requires_grad)

loss_h = MSE()
loss_b = torch.nn.MSELoss()

l_h = loss_h.forward(x, y)
l_b = loss_b(a, y)

l_grad_b = torch.autograd.grad(l_b, a)
l_grad_h = loss_h.backward()
l_grad_b, l_grad_h, 2*(x-y)

True


((tensor([[ 1.5183, -0.3843,  1.1086, -0.7034]]),),
 tensor([[ 1.5183, -0.3843,  1.1086, -0.7034]], grad_fn=<DivBackward0>),
 tensor([[ 6.0733, -1.5371,  4.4345, -2.8134]], grad_fn=<MulBackward0>))

In [70]:
#now check with linear as well

In [71]:
x = torch.randn(5, 9, requires_grad = False)
y = torch.randn(5, 6,requires_grad = False)

linear_h = Linear(9, 6)
linear_b = torch.torch.nn.Linear(9, 6, True)
linear_h.weight.data = linear_b.weight.data
linear_h.bias.data = linear_b.bias.data

output_b = linear_b(x)
l_b = loss_b(output_b, y)
l_b.backward(retain_graph = True)


In [72]:
output_h = linear_h.forward(x)
l_h = loss_h.forward(output_h, y)
linear_h.backward(loss_h.backward())

torch.Size([5, 6, 1])


tensor([[ 0.1647, -0.1573,  0.1060, -0.0011,  0.2459,  0.1486, -0.0902, -0.3420,
         -0.0250],
        [-0.1267,  0.3628,  0.0430,  0.2101, -0.3112, -0.2138,  0.0315,  0.4808,
          0.4700],
        [-0.0755,  0.0731,  0.0260,  0.2532, -0.1287, -0.0775, -0.0555,  0.2201,
          0.2399],
        [-0.1078,  0.2299,  0.2081,  0.0854, -0.1145,  0.0164,  0.1225,  0.1180,
          0.2658],
        [-0.0322,  0.2567,  0.4360,  0.0964, -0.1659,  0.0010,  0.0541,  0.3658,
          0.4664]])

In [73]:
loss_h.backward()

tensor([[-0.0823,  0.4154,  0.2517, -0.1831,  0.2168,  0.4025],
        [ 0.4309, -0.0210,  0.1993,  0.2315, -0.6915, -0.6663],
        [ 0.0111,  0.2427,  0.1193,  0.1924, -0.4485, -0.4428],
        [ 0.2455,  0.0623,  0.3447,  0.1548,  0.0201, -0.4344],
        [ 0.6713,  0.3215,  0.2472,  0.5883, -0.0516, -0.6121]])

In [74]:
linear_b.weight.grad, linear_h.weight.grad,  abs(linear_b.weight.grad - linear_h.weight.grad) < 1e-6

(tensor([[-0.2586,  0.0924,  0.1193, -0.0140,  0.0292,  0.1974, -0.0854,  0.0095,
           0.1938],
         [-0.0937, -0.0808,  0.1225, -0.0528,  0.1437, -0.2777,  0.0057, -0.0760,
           0.0041],
         [-0.2070, -0.0401,  0.0973,  0.0101,  0.1097, -0.1036,  0.0371,  0.0080,
           0.0383],
         [-0.1467,  0.1342,  0.1384, -0.0365, -0.0943,  0.1637, -0.0403,  0.0062,
           0.1348],
         [ 0.2227, -0.0567, -0.0387, -0.1095,  0.2686,  0.0691, -0.2601, -0.0740,
          -0.0696],
         [ 0.3406, -0.2119, -0.1840, -0.0495,  0.2834, -0.2320, -0.1338, -0.1068,
          -0.1774]]),
 tensor([[-0.2586,  0.0924,  0.1193, -0.0140,  0.0292,  0.1974, -0.0854,  0.0095,
           0.1938],
         [-0.0937, -0.0808,  0.1225, -0.0528,  0.1437, -0.2777,  0.0057, -0.0760,
           0.0041],
         [-0.2070, -0.0401,  0.0973,  0.0101,  0.1097, -0.1036,  0.0371,  0.0080,
           0.0383],
         [-0.1467,  0.1342,  0.1384, -0.0365, -0.0943,  0.1637, -0.0403,  0.0062

In [75]:
#Sanity check for whats going on when computing the gradient of the weights
l = 1.*torch.ones(5, 5)*torch.arange(5).view(1, 5)
m = 1.*torch.ones(5, 5)*torch.arange(5).view(1, 5)
l.view(5, -1, 1)*m.view(5, 1, -1)

tensor([[[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]]])

In [76]:
#check that builtin is doing the sum of the two on pytorch
x = torch.randn(2, 9, requires_grad = False)
y = torch.randn(2, 6,requires_grad = False)

In [77]:
linear_b_1 = torch.torch.nn.Linear(9, 6, True)
linear_b_1.weight.data = linear_b.weight.data
linear_b_1.bias.data = linear_b.bias.data

output_b_1 = linear_b_1(x[0])
l_b_1 = loss_b(output_b_1, y[0])
l_b_1.backward(retain_graph = True)

In [78]:
linear_b_2 = torch.torch.nn.Linear(9, 6, True)
linear_b_2.weight.data = linear_b.weight.data
linear_b_2.bias.data = linear_b.bias.data

output_b_2 = linear_b_2(x[1])
l_b_2 = loss_b(output_b_2, y[1])
l_b_2.backward(retain_graph = True)

In [79]:
linear_b_3 = torch.torch.nn.Linear(9, 6, True)
linear_b_3.weight.data = linear_b.weight.data
linear_b_3.bias.data = linear_b.bias.data

output_b_3 = linear_b_3(x)
l_b_3 = loss_b(output_b_3, y)
l_b_3.backward(retain_graph = True)

In [80]:
(linear_b_1.weight.grad + linear_b_2.weight.grad)/2 - linear_b_3.weight.grad 

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.9558e-08,  1.4901e-08,  3.7253e-09, -1.4901e-08, -3.7253e-09,
          0.0000e+00,  5.9605e-08, -6.5193e-09, -7.4506e-09],
        [ 7.4506e-09,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  1.3039e-08, -3.7253e-09,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -7.4506e-09,  0.0000e+00,  0.0000e+00,
         -2.9802e-08, -1.1176e-08,  0.0000e+00, -7.4506e-09],
        [ 0.0000e+00, -3.7253e-09,  1.8626e-09, -7.4506e-09, -1.8626e-09,
         -3.7253e-09, -1.4901e-08,  0.0000e+00,  0.0000e+00],
        [ 8.3819e-09, -1.4901e-08,  3.7253e-09, -2.2352e-08, -7.4506e-09,
         -2.9802e-08, -1.4901e-08, -3.7253e-09,  7.4506e-09]])

In [81]:
(linear_b_1.bias.grad + linear_b_2.bias.grad)/2 -  linear_b_3.bias.grad

tensor([0.0000e+00, 1.4901e-08, 0.0000e+00, 0.0000e+00, 3.7253e-09, 2.9802e-08])

In [82]:
trial = torch.randn(3, 4, 2)
len(trial.squeeze(2).shape)

3

In [83]:
#check using sequential 
x = torch.randn(5,9, requires_grad = False)
y = torch.randn(5,1,requires_grad = False)

#handmade sequential linear + relu 
linear1 = Linear(9, 6, True)
linear2 = Linear(6, 1, True)
sigma1 = Tanh()
sigma2 = Tanh()
loss = MSE()

net = Sequential([
    linear1, 
    sigma1,
    linear2,
    sigma2
])



output = net.forward(x)
loss.forward(output, y)
net.backward(loss.backward())

torch.Size([5, 1, 1])
torch.Size([5, 6, 1])


In [27]:
#intermediate = loss.backward()
#intermediate = sigma2.backward(intermediate)
#intermediate = linear2.backward(intermediate)
#intermediate = sigma1.backward(intermediate)
#intermediate = linear1.backward(intermediate)

In [28]:
#intermediate.shape

In [85]:
linear1_b = torch.nn.Linear(9, 6)
linear1_b.weight.data = linear1.weight.data
linear1_b.bias.data = linear1.bias.data

linear2_b = torch.nn.Linear(9, 6)
linear2_b.weight.data = linear2.weight.data
linear2_b.bias.data = linear2.bias.data



model_b = torch.nn.Sequential(
            linear1_b,
            torch.nn.Tanh(),
            linear2_b,
            torch.nn.Tanh()
        )
output = model_b(x)
l = torch.nn.MSELoss()(model_b(x), y)
l.backward()

In [86]:
(abs(linear1.weight.grad - linear1_b.weight.grad) < 1e-7).all()

tensor(True)

In [87]:
(abs(linear1.bias.grad - linear1_b.bias.grad)< 1e-7).all()

tensor(True)

In [88]:
(abs(linear2.weight.grad - linear2_b.weight.grad)< 1e-7).all()

tensor(True)

In [90]:
(abs(linear2.bias.grad - linear2_b.bias.grad)< 1e-6).all()

tensor(True)